# Django Object Relational Mapper (ORM)

## Introduction

This notebook provides a brief overview of the [Django](https://docs.djangoproject.com/en/3.1/) [ORM](https://en.wikipedia.org/wiki/Object%E2%80%93relational_mapping), which allows us to access the database using Python objects. We will assume you have some working knowledge of Python, databases ([PostgreSQL](https://www.postgresql.org/)) and [object-oriented programming](https://pythonguides.com/object-oriented-programming-python/). New concepts relating to the Django ORM will be introduced as we work through the notebook.

The objective of the notebook is to get you familiar with Python commands that you can use for accessing the data. This includes: 

* How data is stored
* How to access the data
* How to query the data
* How to perform analysis on data.

## Initial Setup

To get started, run the cell below which will configure our Python environment to have access to Django shell functionality. **You must run this cell before any other cells**.

In [14]:
# Setup environment
import sys
import os
import django
sys.path.append('../')
sys.path.append('../src/')
django.setup()

In [16]:
# Configurations for this notebook.
os.environ["DJANGO_DATABASE_ENGINE"]="django.db.backends.mysql"
os.environ["DJANGO_DATABASE_NAME"]="test"
os.environ["DJANGO_DATABASE_USER"]="test"
os.environ["DJANGO_DATABASE_PASSWORD"]="test"

## Data storage

The data is stored in a PostgreSQL database. This database has a certain [schema](https://en.wikipedia.org/wiki/Database_schema) which is determined by administrators to structure the data that is being stored. For the WALLABY database, we have a pre-defined schema that contains the tables for storing detections, sources, runs, comments etc. 

## Models

The most important concept in the ORM is the [Django Model](https://docs.djangoproject.com/en/3.1/topics/db/models/). Models are a pythonic representation of a database table. They contain information about the name and data type for each field in a table. To use a 

```

```

This [page](https://docs.djangoproject.com/en/3.1/topics/db/models/) provides an in-depth reference for Django Models that you can read for more information.

## Queries

## Query logic